## Model Project - The Bertrand Model Hackathon

Imports and set magics:

In [104]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
from modelproject import Bertrand
from modelproject import BertrandN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1 Model description

I start by considering a duopolic economy consisting of 2 identical firms $i=1,2$ competing in a market characterized by perfect competition and a homogenous good. 

The firms set their prices $p_i$ simultaneously, 

The firms have an identical, constant marginal cost $c$. 

Demand is a downward sloping function of the prices set and maximum willingness to pay, $D=a-p$, or for a single firm: 

$$D(p_i ) = a - p_i \ if\ p_i < p_j$$
$$D(p_i) = (a-p_i)/2 \ if\ p_i=p_j$$
$$D(p_i) = 0 \ if\ p_i > p_j$$ 

Profit becomes: 
 $$\pi_i = (p_i-c)D(p_i)$$








Since firm $i$ can overtake the entire market by undercutting firm $j$, $p_i$ will converge to $c$, given $a>c$ 

### 2 Numerical solution

#### 2.1 Algorithm
The model is solved (and markets cleared) by:
1. Initialize by defining parameter values of a, c, and the number of simulation iterations
2. Define the demand and profit function according to the section above 
3. Find the Best Response (BR) function of firm $i$, which is the price $p_i$ that maximizes profit given the price of firm $j$ 
4. Find the BR function of firm $j$
5. Find the Nash equilibrium by computing the set of BR-values that solves both 3. and 4., or ensure the convergence of BR values
6. Calculate profits and quantity

#### 2.2 Testing the solver

I initialize with the parameters a = 10 and c = 1, and then a = 10 and c = 11 :

In [62]:
Bertrand0  = Bertrand(10,1,1)
temp0 = Bertrand0.solve()


# Printing the extracted values
print('p1:', temp0[0])
print('p2:', temp0[1])
print('profit1:', temp0[2])
print('profit2:', temp0[3])
print('q1:', temp0[4])
print('q2:', temp0[5])


p1: 1.0
p2: 1.0
profit1: 0.0
profit2: 0.0
q1: 4.5
q2: 4.5


In [106]:
Bertrand1  = Bertrand(10,11,11)
temp1 = Bertrand1.solve()


# Printing the extracted values
print('p1:', temp1[0])
print('p2:', temp1[1])
print('profit1:', temp1[2])
print('profit2:', temp1[3])
print('q1:', temp1[4])
print('q2:', temp1[5])

p1: 11
p2: 11
profit1: 0.0
profit2: 0.0
q1: 0
q2: 0


Let's see if it can converge correctly with more awkward numbers:

In [107]:
Bertrand1  = Bertrand(15.2,4.6,2.2)
temp2 = Bertrand1.solve()


# Printing the extracted values
print('p1:', temp2[0])
print('p2:', temp2[1])
print('profit1:', temp2[2])
print('profit2:', temp2[3])
print('q1:', temp2[4])
print('q2:', temp2[5])

p1: 4.6
p2: 4.599992675781255
profit1: 0
profit2: 25.439939941352645
q1: 0
q2: 10.600007324218744


Firm 2 undercuts by $\epsilon$ to secure the whole market. Class looks good.

#### 2.3 Solution as a function of parameters 
Let's try and see what happens when the firms face different marginal costs and maximum willingness to pay. 

In [100]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact

def interactive_plot(a, c1, c2):
    bertrand = Bertrand(a, c1, c2)
    p1, p2, profit1, profit2, d1, d2 = bertrand.solve()
    
    total_price = min(p1, p2)
    total_quantity = a - total_price
    
    aggregate_prices = np.linspace(0, a, 100)
    aggregate_quantities = a - aggregate_prices

    plt.figure(figsize=(8, 6))
    plt.plot(aggregate_quantities, aggregate_prices, label='Aggregate Q as function of P')
    plt.scatter(total_quantity, total_price, color='red', label='Equilibrium')
    plt.text(total_quantity, total_price, f'({total_quantity:.2f}, {total_price:.2f})', ha='left', va='bottom')
    
    plt.xlabel('Aggregate Quantity')
    plt.ylabel('Aggregate Price')
    plt.title('Aggregate Price vs Quantity')
    plt.legend()
    
    # Annotate each firm's profit, price, and quantity
    plt.annotate(f'Firm 1:\nProfit = {profit1:.2f}\nPrice = {p1:.2f}\nQuantity = {d1:.2f}',
                 xy=(1.05, 0.95), xycoords='axes fraction', fontsize=10, ha='left', va='top')
    plt.annotate(f'Firm 2:\nProfit = {profit2:.2f}\nPrice = {p2:.2f}\nQuantity = {d2:.2f}',
                 xy=(1.05, 0.65), xycoords='axes fraction', fontsize=10, ha='left', va='bottom')
    
    plt.show()

interact(interactive_plot, a=(1, 10, 0.1), c1=(0, 10, 0.1), c2=(0, 10, 0.1));

interactive(children=(FloatSlider(value=5.0, description='a', max=10.0, min=1.0), FloatSlider(value=5.0, descr…

### 3 Expansion to multiple firms

We have $n$ ($i=1,2,...n$) firms competing in a market characterized by perfect competition and a homogenous good,

firms set their prices $p_1, p_2, ..., p_n$ simultaneously,

firms face identical marginal costs $c_1 = c_2 =...=c_n$,

demand is a downwards sloping function of price and MWTP; 

$$D(p_i) = a - p_i \ if \ p_i < p_1, p_2,...,p_n$$ 
$$D(p_i) = (a- \frac{p_i}{k}) \ if\ p_i = p_j, p_l,...,p_k$$
$$D(p_i) = 0 \ if\ p_i > p_j$$

#### 3.1 Numerical solution

We start out by testing with +1 firms:

In [108]:
a = 10
costs = [1, 1, 1]  #marginal cost vector 
bertrand_model = BertrandN(a, costs)
prices, profits, demands = bertrand_model.solve()

print("Prices:", prices)
print("Profits:", profits)
print("Demands:", demands)

Prices: [1, 1, 1]
Profits: [0.0, 0.0, 0.0]
Demands: [3.0, 3.0, 3.0]


In [111]:
a1 = 5
costs1 = [1, 2, 3]  
bertrand_model1 = BertrandN(a1, costs)
prices1, profits1, demands1 = bertrand_model1.solve()

print("Prices:", prices1)
print("Profits:", profits1)
print("Demands:", demands1)

Prices: [1.9999023437500032, 2.0, 3.0]
Profits: [2.9998046779632634, 0.0, 0.0]
Demands: [3.000097656249997, 0, 0]


Looks good so far. Let's try to up the ante:

In [130]:
a2 = 10 
c_min = 1  # minimum cost
c_max = 10  # maximum cost

costs2 = np.random.uniform(c_min, c_max, 100) #100 firms with random marginal costs in da house
bertrand_model2 = BertrandN(a2, costs2)
prices2, profits2, demands2 = bertrand_model2.solve()

print("Prices:", prices2)
print("Profits:", profits2)
print("Demands:", demands2)

Prices: [4.831576648880567, 1.931624070982823, 8.526302729873809, 5.5552239315225975, 4.204381398755786, 6.7886061630233865, 6.004526607344169, 7.794288417709207, 5.445282764522666, 2.3356269670485172, 1.193737393140753, 8.542986618772776, 5.360122207182864, 3.6995054110995347, 9.751109307857037, 3.3968963876926987, 2.77237414595242, 2.6511843579642136, 2.944304413867709, 5.421716142334781, 3.3181780176051747, 8.558612035941625, 7.817142524548094, 5.351477224530667, 1.9064386046212616, 2.279940818680644, 7.604260264736011, 7.742524120931221, 9.24404505928381, 7.806627474942223, 4.15728425245786, 2.7678899450051744, 6.092620170751931, 4.055552893290899, 6.983592116984831, 1.1758064942373703, 3.56265403048713, 7.266220793811998, 2.9447855882890472, 7.412475005291934, 7.093967880136261, 9.83155926886189, 8.064380916372482, 5.406129803762436, 9.01309217994917, 7.192590598276314, 9.412160197605843, 4.801813465070669, 1.4846163970669575, 4.965812910629969, 4.942422565730166, 1.70729447714362

This is a bit much, luckily we have a new function that finds the market clearing price:

In [134]:

prices3, profits3, demands3 = bertrand_model2.solvemin()

print("Pricemin:", prices3)
print("Profit:", profits3)
print("Demand:", demands3)

Pricemin: 1.1758064942373703
Profit: 0.16723091072648
Demand: 8.82419350576263


### 4 Conclusion

Add concise conclusion. 